#  ATENA-TF Live Recommendations System

This notebook provides **real-time AI recommendations** while you explore data!

## How it works:
1. 🧠 **Trained model** suggests the next best action
2.  **You decide** whether to follow or choose your own action
3.  **Instant results** show filtered/grouped data
4.  **Cycle continues** - AI learns from your choices

---


In [1]:
#  Import required libraries
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import numpy as np

# Import our TF Recommender Agent
from live_recommender_agent import TFRecommenderAgent, find_latest_trained_model
import gym_atena.global_env_prop as gep
from gym_atena.lib.helpers import (
    PRETTY_STR_MAP_ATENA_INT_OPERATOR,
    INVERSE_AGG_MAP_ATENA,
)

print(" All imports successful!")


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


Registering ATENAcont-v0 environment
Configuration loaded with:
  - humanity_coeff: 1.0
  - diversity_coeff: 2.0
  - kl_coeff: 1.5
  - compaction_coeff: 2.0
  - adam_lr: 0.0003
  - ppo_gamma: 0.995
  - ppo_lambda: 0.97
✅ Using Snorkel compatibility adapter
Configuration loaded with:
  - humanity_coeff: 1.0
  - diversity_coeff: 2.0
  - kl_coeff: 1.5
  - compaction_coeff: 2.0
  - adam_lr: 0.0003
  - ppo_gamma: 0.995
  - ppo_lambda: 0.97
✅ REWARD STABILIZER: DISABLED (stable mode like train_ipdate-1009-18:54.png)
✅ All imports successful!


##  Initialize the Recommender System


In [2]:
# Find and load the latest trained model
model_path = find_latest_trained_model()

if model_path:
    print(f" Found trained model: {model_path}")
    
    # Create the recommender agent
    recommender_agent = TFRecommenderAgent(
        model_path=model_path,
        dataset_number=0,  # Start with dataset 0
        schema='NETWORKING'  # Default to networking data
    )
    
    print("\n Recommender system initialized!")
    print(f" Dataset shape: {recommender_agent.original_dataset.shape}")
    print(f"📋 Columns: {list(recommender_agent.original_dataset.columns)}")
    
else:
    print(" No trained model found!")
    print("Please run training first: python main.py --episodes 100")


✅ Found latest model: results/0511-resumed/best_agent (best_agent)
📂 Found trained model: results/0511-resumed/best_agent
🔄 Loading datasets for schema: NETWORKING






































































































































































































































































































































































































































































































































































































































































































































































































































































/Users/edenmironi/Desktop/school/third year/Atena/ATENA-A-EDA/Compare/.venv/lib/python3.12/site-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/Users/edenmironi/Desktop/school/third year/Atena/ATENA-A-EDA/Compare/atena-tf 2/gym_atena/envs/atena_snorkel/snorkel_compatibility.py:122: DeprecationWarning: Please import `csr_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.csr` namespace is deprecated and will be removed in SciPy 2.0.0.
  return super().find_class(module, name)
INFO:root:Computing O...
INFO:root:Estimating \mu...


✅ Datasets loaded successfully!
🔧 Fixing old snorkel.learning references in checkpoint...
✅ Successfully loaded Snorkel checkpoint with compatibility fixes
🔧 Initializing real LabelModel with checkpoint data...
   Fitting LabelModel with dummy data: L_train(100, 51), class_balance=[0.5, 0.5]


INFO:root:[0 epochs]: TRAIN:[loss=119.091]
INFO:root:[10 epochs]: TRAIN:[loss=42.754]
INFO:root:[20 epochs]: TRAIN:[loss=20.931]
INFO:root:[30 epochs]: TRAIN:[loss=9.639]
INFO:root:[40 epochs]: TRAIN:[loss=5.120]
INFO:root:[50 epochs]: TRAIN:[loss=4.729]
INFO:root:[60 epochs]: TRAIN:[loss=4.783]
INFO:root:[70 epochs]: TRAIN:[loss=4.620]
INFO:root:[80 epochs]: TRAIN:[loss=4.519]
INFO:root:[90 epochs]: TRAIN:[loss=4.497]
INFO:root:Finished Training


✅ Real LabelModel initialized and ready for predictions!
✅ Loaded Snorkel model from snorkel_checkpoints
Enhanced ATENA Environment initialized with:
  - Rule-based humanity scoring: ✓
  - Enhanced diversity rewards: ✓
  - Detailed reward tracking: ✓
  - Max steps: 12
🎯 Initializing ParamSoftmaxPolicy (master's SUCCESS architecture - 600 channels!)
✅ Using environment parametric segments: ((), (12, 3, 26), (12,))
✅ FFParamSoftmax network built with 2 layers x 600 units
🎯 FFParamSoftmax Policy initialized:
  - Hidden channels: 600 (master's 600)
  - Parametric segments: ((), (12, 3, 26), (12,))
  - Pre-output size: 56
  - Beta (temperature): 1.0
🏗️  Building networks...
✅ Networks built - Policy vars: 6, Value vars: 6
✅ Policy weights loaded from results/0511-resumed/best_agent (.weights.h5 format)
✅ Value weights loaded from results/0511-resumed/best_agent (.weights.h5 format)
⚠️ Failed to load model: 'NoneType' object has no attribute 'mean'
⚠️ No compatible trained model found at res

## 🎛️ Interactive Control Panel


In [3]:
# Create output area for results
output_area = widgets.Output()

# Recommendation display
recommendation_display = widgets.HTML(
    value="<b><font color='blue'> Getting first recommendation...</font></b>"
)

def show_recommendation():
    """Update recommendation display"""
    try:
        recommendation_str = recommender_agent.get_agent_action_str().replace("<", "").replace(">", "")
        recommendation_display.value = f"<b><font color='red'> AI Recommendation: {recommendation_str}</font></b>"
    except Exception as e:
        recommendation_display.value = f"<b><font color='orange'> Recommendation error: {str(e)}</font></b>"

# Show initial recommendation
if 'recommender_agent' in locals():
    show_recommendation()


In [4]:
# Action controls - גרסה פשוטה ומהירה
action_type_dropdown = widgets.Dropdown(
    options=['Group', 'Filter', 'Back'],
    value='Group',
    description='Action Type:',
    disabled=False,
    layout={'width': '200px'}
)

# עמודות קבועות עבור networking data
column_dropdown = widgets.Dropdown(
    options=['packet_number', 'eth_src', 'eth_dst', 'tcp_srcport', 'tcp_dstport'],
    value='eth_dst',  # ברירת מחדל חכמה
    description='Column:',
    disabled=False,
    layout={'width': '200px'}
)

filter_operator_dropdown = widgets.Dropdown(
    options=['CONTAINS', 'EQUAL', 'NOT EQUAL'],
    value='CONTAINS',
    description='Filter Op:',
    disabled=False,
    layout={'width': '200px'}
)

filter_term_text = widgets.Text(
    placeholder='e.g. 00:0c:29:54:bf:79',
    description='Filter term:',
    disabled=False,
    layout={'width': '250px'}
)

print(" UI Controls created successfully!")
print("📋 Available columns: packet_number, eth_src, eth_dst, tcp_srcport, tcp_dstport")

✅ UI Controls created successfully!
📋 Available columns: packet_number, eth_src, eth_dst, tcp_srcport, tcp_dstport


In [5]:
# Action buttons
apply_custom_button = widgets.Button(
    description='Apply My Action',
    button_style='info',
    layout={'width': '150px'}
)

apply_ai_button = widgets.Button(
    description='Apply AI Recommendation',
    button_style='success', 
    layout={'width': '200px'}
)

reset_button = widgets.Button(
    description='Reset',
    button_style='warning',
    layout={'width': '100px'}
)


In [6]:
# Helper functions
def get_ui_action_vector():
    """Convert UI selections to action vector"""
    action_type = action_type_dropdown.value
    
    if action_type == "Back":
        return [0] * 6
    elif action_type == "Filter":
        # Get column index
        try:
            col_name = column_dropdown.value
            col = list(recommender_agent.original_dataset.columns).index(col_name)
        except (ValueError, KeyError):
            col = 0  # Fallback to first column
        
        condition = PRETTY_STR_MAP_ATENA_INT_OPERATOR.get(filter_operator_dropdown.value, 6)
        term = 0  # dummy term
        return [1] + [col] + [condition] + [term] + [0, 0]
        
    elif action_type == "Group":
        # Get column index
        try:
            col_name = column_dropdown.value
            col = list(recommender_agent.original_dataset.columns).index(col_name)
        except (ValueError, KeyError):
            col = 0  # Fallback to first column
        
        agg_col = 0  # Default aggregation column
        agg_func = INVERSE_AGG_MAP_ATENA.get('count', 1)  # count is default
        return [2] + [col] + [0, 0] + [agg_col] + [agg_func]
    
    return [0] * 6  # Fallback

def apply_custom_action_handler(button):
    """Handle custom action application"""
    action_vec = get_ui_action_vector()
    
    with output_area:
        clear_output(wait=True)
        print(f" Applying custom action: {action_vec}")
        
        try:
            step_result = recommender_agent.apply_custom_action(action_vec, filter_term_text.value)
            
            if step_result.df_to_display is not None:
                print(f" Result ({len(step_result.df_to_display)} rows):")
                display(step_result.df_to_display.head(10))
            else:
                print(" No data to display")
                
        except Exception as e:
            print(f" Error: {e}")
    
    show_recommendation()

def apply_ai_action_handler(button):
    """Handle AI recommendation application"""
    with output_area:
        clear_output(wait=True)
        print(f" Applying AI recommendation...")
        
        try:
            step_result = recommender_agent.apply_agent_action(use_last_recommendation=True)
            
            if step_result.df_to_display is not None:
                print(f" Result ({len(step_result.df_to_display)} rows):")
                display(step_result.df_to_display.head(10))
            else:
                print(" No data to display")
                
        except Exception as e:
            print(f" Error: {e}")
    
    show_recommendation()

def reset_handler(button):
    """Handle reset"""
    with output_area:
        clear_output(wait=True)
        print(" Resetting environment...")
        
        try:
            recommender_agent.reset_environment()
            print(" Reset complete!")
            display(recommender_agent.original_dataset.head(10))
        except Exception as e:
            print(f" Reset error: {e}")
    
    show_recommendation()

# Connect button handlers
apply_custom_button.on_click(apply_custom_action_handler)
apply_ai_button.on_click(apply_ai_action_handler)
reset_button.on_click(reset_handler)

print(" UI handlers configured!")


✅ UI handlers configured!


## 🎮 Live Recommendations Interface

### Instructions:
1.  **AI Recommendation** appears below
2.  **Your choice**: Follow AI or choose your own action
3.  **Results** appear in the output area
4.  **New recommendation** updates automatically

---


In [7]:
# Display the complete interface
control_panel = widgets.VBox([
    widgets.HTML("<h3>🎛️ Control Panel</h3>"),
    
    # Recommendation display
    recommendation_display,
    
    widgets.HTML("<hr>"),
    
    # Controls row 1: Action type and column
    widgets.HBox([
        action_type_dropdown, 
        column_dropdown
    ]),
    
    # Controls row 2: Filter options
    widgets.HBox([
        filter_operator_dropdown, 
        filter_term_text
    ]),
    
    widgets.HTML("<br>"),
    
    # Action buttons
    widgets.HBox([
        apply_custom_button,
        apply_ai_button, 
        reset_button
    ]),
    
    widgets.HTML("<hr>")
])

display(control_panel)


In [8]:
# Output area for results
display(widgets.HTML("<h3> Results</h3>"))
display(output_area)


HTML(value='<h3>📊 Results</h3>')

Output()

In [9]:
# Initialize display with original dataset
if 'recommender_agent' in locals():
    with output_area:
        clear_output(wait=True)
        print("🗂️ Original Dataset:")
        print(f"Shape: {recommender_agent.original_dataset.shape}")
        display(recommender_agent.original_dataset.head(10))
        
    # Show the first recommendation
    show_recommendation()


##  Usage Tips

###  Best Practices:
- **Follow AI suggestions** to see what the model learned
- **Try your own actions** to explore different paths
- **Use Back action** frequently to compare different views
- **Reset environment** to start fresh analysis

###  Understanding Recommendations:
- **Filter actions**: Narrow down data to specific subsets
- **Group actions**: Aggregate data by columns
- **Back actions**: Navigate between different views

### 🧠 What the AI Learned:
Based on training, the model prefers:
1. **eth_dst filtering** - identifies traffic destinations
2. **tcp_srcport grouping** - analyzes port usage patterns 
3. **Frequent navigation** - thorough data exploration

---

##  Enjoy exploring your data with AI guidance!
